### Advent of Code 2016, Day 13

This is supposed to be solvable by breadth-first search according
to the [classifier](). Let's see if it'll make a good example.

In [147]:
from collections import deque
from dataclasses import dataclass
from typing import Optional, Iterator

In [148]:
_ADJACENT_DELTA = [(-1, 0), (1, 0), (0, -1), (0, 1)]

@dataclass(unsafe_hash=True)
class Point:
    x: int
    y: int

    def get_adjacent(self) -> Iterator["Point"]:
        for (dx, dy) in _ADJACENT_DELTA:
            new_x, new_y = self.x + dx, self.y + dy
            if new_x >= 0 and new_y >= 0:        
                yield Point(new_x, new_y)

In [149]:
FAV_NUMBER = 1350
START = Point(1, 1)
GOAL = Point(31, 39)

In [150]:
def is_open(p: Point, fav_number: int=FAV_NUMBER) -> bool:
    s = fav_number + p.x**2 + 3*p.x + 2*p.x*p.y + p.y + p.y**2
    return bin(s).strip("0b").count('1') % 2 == 0

In [151]:
def open_adjacent(p: Point, fav_number: int=FAV_NUMBER) -> Iterator["Point"]:
    for adjacent in p.get_adjacent():
        if is_open(adjacent, fav_number=fav_number):
            yield adjacent

In [164]:
def visualize(levels: int=10, fav_number: int=FAV_NUMBER):
    for y in range(levels):
        line = ['.' if is_open(Point(x, y), fav_number=fav_number) else '#' for x in range(levels)]
        print("".join(line))

Let's check if it matches test input:

In [153]:
visualize(10, fav_number=10)

.#.####.##
..#..#...#
#....##...
###.#.###.
.##..#..#.
..##....#.
#...##.###
.##..#.##.
#.###....#
###.####.#


And for our input the maze looks like this:

In [154]:
visualize(40)

##.######.##.####...#.##....#..#.#..#.##
.....##.#..#.##.#.#.######..#.##..#.####
###....###......#..#..#..####.###..#..#.
#.####.####..###.#.##.#.#...#..#.#.##.##
........######.###..###..##.#..###..##.#
#..##.#..##..#.......###.#..###......#.#
###.#........##.##.#.....#....#.##.#.###
..##.######.#.#.##...###.##...##.#..#..#
.#.##....##..##.#.###..#.###.#.##.#..#..
.##.#..#.###.#..#...#.##...#..#.#..#....
..######..#..#.######.#####.#..###..##.#
....#.....##.#.##..#...##.#.##.####..#.#
.##.#.##...#.......#......#.....#.###...
.#..#.###.#######.########.##.#.###.##..
.#..#..##.#...###.##...#.##.#..#.....##.
.###.#.##.#.....#....#..#.##.#.####.#.#.
.#.###...###.##.#.###.#.##.###...##..##.
.#.....#..##.#..##..##...#.....#..##.#..
.#..#####....#...###.#...##.#####.#..#..
###....#.##.###.#..#.##.#.###..#..##.###
#.###..####..##..#.#.##..#.....#...#.#..
###.#....#.#...#..##..##...###.##.##..#.
....###..##.##..#.###.#.####.#.##.###.#.
.####.##..#######..#..#.....##.....#..##
.#.....###..#...

Time for the breadth-first search: since it will go level-by-level, in this case: 
all nodes checked at a given graph layer will be on the equal distance from the "entrance" at `(1, 1)`.

In [155]:
def bfs(fav_number: int=FAV_NUMBER, 
        start: Optional[Point]=None,
        goal: Optional[Point]=None) -> dict[Point, Point]:
    """
    A breadth-first implementation using deque as a queue.
    
    Starts at `start`, looks for `goal`, uses `fav_number` to figure out 
    if there are walls / node connectivity.

    Returns a dict mapping each node to its "parent",
    i.e. the node from which it can be reached on the shortest
    path from `start`.
    """
    start = START if start is None else start
    goal = GOAL if goal is None else goal

    parents = {}
    queue, explored = deque([start]), set([start])
    while queue:
        curr = queue.popleft()

        if curr == goal:
            return parents

        for next in open_adjacent(curr, fav_number):
            if next not in explored:
                explored.add(next)
                parents[next] = curr
                queue.append(next)


In [156]:
def reconstruct_path(parents: dict[Point, Point], 
                     start: Optional[Point]=None,
                     goal: Optional[Point]=None) -> list[Point]:
    start = START if start is None else start
    goal = GOAL if goal is None else goal

    rev = [goal]
    curr = goal
    while curr in parents:
        parent = parents[curr]
        rev.append(parent)
        curr = parent
    
    rev.reverse()
    return rev

This is enough to solve p1!

In [157]:
def p1(fav_number: int=FAV_NUMBER, goal: Optional[Point]=None) -> int:
    parents = bfs(fav_number=fav_number, goal=goal)
    p1_path = reconstruct_path(parents, goal=goal)
    return len(p1_path) - 1

In [158]:
assert p1(fav_number=10, goal=Point(7, 4)) == 11
assert p1() == 92

We'll just copy and slightly modify the `bfs` function above for p2.

In [159]:
def p2(max_steps: int=50,
       fav_number: int=FAV_NUMBER, 
       start: Optional[Point]=None) -> set[Point]:
    """
    Another breadth-first implementation, based on the `bfs`, but
    using `max_steps` as a termination condition.

    Instead of returning `parents`, it uses keys & values in that dict
    to compute all visited nodes and returns a corresponding set.
    """
    start = START if start is None else start

    parents = {}
    queue, explored = deque([(start, 0)]), set([start])
    while queue:
        (curr, step) = queue.popleft()

        if step == max_steps:
            return set(parents.keys()).union(set(parents.values()))

        for next in open_adjacent(curr, fav_number):
            if next not in explored:
                explored.add(next)
                parents[next] = curr
                queue.append((next, step + 1))


In [160]:
assert Point(7, 4) in p2(11, fav_number=10)
assert Point(7, 5) in p2(11, fav_number=10)
assert Point(7, 4) not in p2(10, fav_number=10)
assert Point(7, 5) in p2(10, fav_number=10)

In [161]:
assert len(p2(50)) == 124

### Upshot

It seems like a good and small enough example for a breadth-first usage.

Especially part 2 is interesting, since it showcases two important properties:

- bfs can be used to traverse a graph "level-by-level"
- Dijkstra's etc. use bfs's quality of always going in an order of path shortness (i.e., shorter-path-to nodes
will be evaluated before longer-path-to nodes).

Additionally, we can add a nice visualization.

In [191]:
def visualize_path(path: list[Point], fav_number: int=FAV_NUMBER):    
    def _show(x, y):
        p = Point(x, y)
        if is_open(p, fav_number=fav_number):
            if p in path:
                if p == path[0]:
                    return "S"
                elif p == path[-1]:
                    return "G"
                else:
                    return "O"
            else:
                return "."
        else:
            return "#"

    max_x, max_y = max([point.x for point in path]), max([point.y for point in path])
    for y in range(max_y + 2):
        line = [_show(x, y) for x in range(max_x + 2)]
        print("".join(line))

In [192]:
visualize_path(
    reconstruct_path(
        bfs(fav_number=10, goal=Point(7, 4)), 
        goal=Point(7, 4)), 
    fav_number=10
)

.#.####.#
.S#..#...
#OOO.##..
###O#.###
.##OO#.G#
..##OOOO#
#...##.##


In [193]:
visualize_path(reconstruct_path(bfs()))

##.######.##.####...#.##....#..#.#..#.##.
.SOOO##.#..#.##.#.#.######..#.##..#.####.
###.OOO###......#..#..#..####.###..#..#.#
#.####O####..###.#.##.#.#...#..#.#.##.##.
......OO######.###..###..##.#..###..##.#.
#..##.#OO##..#.......###.#..###......#.##
###.#...OOOO.##.##.#.....#....#.##.#.###.
..##.######O#.#.##...###.##...##.#..#..##
.#.##....##OO##.#.###..#.###.#.##.#..#..#
.##.#..#.###O#..#...#.##...#..#.#..#.....
..######..#.O#.######.#####.#..###..##.##
....#.....##O#.##..#...##.#.##.####..#.##
.##.#.##...#OOOOOO.#......#.....#.###...#
.#..#.###.#######O########.##.#.###.##...
.#..#..##.#...###O##OOO#.##.#..#.....##..
.###.#.##.#.....#OOOO#OO#.##.#.####.#.#.#
.#.###...###.##.#.###.#O##.###...##..##.#
.#.....#..##.#..##..##.O.#.....#..##.#..#
.#..#####....#...###.#.O.##.#####.#..#..#
###....#.##.###.#..#.##O#.###..#..##.####
#.###..####..##..#.#.##OO#OOOOO#...#.#..#
###.#....#.#...#..##..##OOO###O##.##..#..
....###..##.##..#.###.#.####.#O##.###.#..
.####.##..#######..#..#.....##OOOO